In [1]:
import pandas as pd
import os
import re
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
#Datasets
## https://www.kaggle.com/datasets/theatticusproject/atticus-open-contract-dataset-aok-beta
## https://www.kaggle.com/datasets/shivamb/legal-citation-text-classification

In [2]:
import json
import pprint

data = json.load(open('legal/cuad_atticus/CUADv1.json', 'rb'))
#pprint.pprint(data, depth=2)

In [3]:
#data = json.load(open('legal/cuad_atticus/train_separate_questions.json', 'rb'))

In [4]:
reduced_legal_list = []
for i in range(len(data['data'])):
    context_data = data['data'][i]['paragraphs'][0]['context']
    data_keep = [x.strip() for x in context_data.split('\n') if len(x.strip()) > 350]
    reduced_legal_list += data_keep

In [5]:
legal_text_classification_df = pd.read_csv('legal/legal_text_classification.csv')

In [6]:
#subset
legal_text_classification_df.drop_duplicates(subset=['case_text'],inplace=True)

In [7]:
legal_text_classification_df

,case_id,case_outcome,case_title,case_text
0,Case1,cited,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,cited,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
4,Case5,cited,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...
5,Case6,cited,GEC Marconi Systems Pty Ltd v BHP Information ...,I accept that the making of a rolled up offer ...
7,Case8,cited,Seven Network Limited v News Limited (2007) 24...,On the question of the level of unreasonablene...
...,...,...,...,...
24969,Case25192,cited,Buckley v Bennell Design &amp; Constructions P...,Those cases reflect the policy underlying s 13...
24975,Case25198,cited,Newtons Travel Services Pty Ltd v Ansett Trans...,"In my view, it is clear that the Court may do ..."
24979,Case25202,cited,Ratepayers &amp; Residents Action Association ...,Once the threshold prescribed by s 1335 is sat...
24983,Case25206,distinguished,"Tullock Ltd v Walker (Unreported, Supreme Cour...",Given the extent to which Deumer stands to gai...


In [8]:
legal_text_classification_df.dropna(subset=['case_text'],inplace=True)

In [9]:
legal_text_classification_df['length'] = legal_text_classification_df['case_text'].str.len()

In [10]:
legal_text_classification_df['length'].describe()

count     17920.000000
mean       2551.235938
std        4433.934383
min          95.000000
25%         890.000000
50%        1589.000000
75%        2889.250000
max      133561.000000
Name: length, dtype: float64

In [11]:
legal_text_df = legal_text_classification_df[(legal_text_classification_df.length > 500) & (legal_text_classification_df.length < 2754)]

In [12]:
legal_text_df.case_text.iloc[15]

"On 30 March 2006 Siopis J made interim orders restraining the defendants from removing any of their property from Australia or from otherwise dealing with it except to the extent set out in the order. He also directed the surrender of their passports. His Honour published his reasons for those orders on 5 April 2006 Australian Securities and Investments Commission, In the Matter of Richstar Enterprises Pty Ltd (ACN 099 071 968) v Carey [2006] FCA 366. 16 On 7 April 2006 I made orders extending the operation of the interim injunctions granted by Siopis J (subject to some undisputed variations) until 5pm on 12 April 2006 which was the date set down for the hearing of the substantive application. Following the hearing of the substantive application those orders were further extended to 5pm on Thursday 20 April 2006 subject to further agreed variations. Judgment on the substantive application was reserved to 2.15pm on Thursday 20 April 2006. 17 There was an application made on behalf of t

In [13]:
new_legal_list = reduced_legal_list + legal_text_df.case_text.tolist()

In [14]:
len(new_legal_list)

31672

In [16]:
train, val = train_test_split(new_legal_list, test_size=0.15, random_state=42)

In [21]:
with open('legal/legal_dataset/legal_train.txt', 'w') as f:
    for file in train:
        f.write(f"{file}\n")
        
with open('legal/legal_dataset/legal_val.txt', 'w') as f:
    for file in val:
        f.write(f"{file}\n")